In [1]:
import json
from utils.OpenMeteoWeather import OpenMeteoWeather
from utils.Copernicus import AdvancedCopernicus
from utils.PlanetPositions import PlanetPositions
import pandas as pd
from utils.frost_server import FrostServer
import time
from datetime import datetime



In [2]:
output_filename = "output.nc"
start_datetime = "2023-09-01T00:00:00Z"
end_datetime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Extrahiere nur das Datum
end_date = datetime.strptime(end_datetime.split(" ")[0], "%Y-%m-%d").date()

latitude = 54.3323
longitude = 10.1519
val_range = 0.5

minimum_latitude = latitude - val_range
maximum_latitude = latitude + val_range
minimum_longitude = longitude - val_range
maximum_longitude = longitude + val_range

In [3]:
end_date_str = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")

print(end_date_str.split("T")[0])

2025-02-18


In [4]:
# convert datetime to YYYY-MM-DD
start_datetime = start_datetime.split("T")[0]
end_datetime = end_datetime.split("T")[0]

omw = OpenMeteoWeather(latitude=latitude, 
                       longitude=longitude, 
                       start_date=start_datetime, 
                       end_date=end_date_str.split("T")[0])
df_omw = omw.get_weather_dataframe()

In [5]:
df_omw

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,...,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_to_1cm,soil_moisture_1_to_3cm,soil_moisture_3_to_9cm,soil_moisture_9_to_27cm,soil_moisture_27_to_81cm
0,2025-02-18 22:00:00+00:00,-2.022000,77.000000,-5.515003,-5.313011,0.0,0.0,0.0,0.0,0.0,...,-0.622,-1.472,-0.172,0.028,2.578,0.282,0.284,0.286,0.299,0.321
1,2025-02-18 21:00:00+00:00,-1.522000,74.000000,-5.553801,-4.849452,0.0,0.0,0.0,0.0,0.0,...,-0.622,-1.022,-0.122,0.028,2.578,0.282,0.284,0.286,0.299,0.321
2,2025-02-18 20:00:00+00:00,-0.972000,71.000000,-5.568116,-4.312119,0.0,0.0,0.0,0.0,0.0,...,-0.622,-0.572,-0.122,0.028,2.628,0.283,0.284,0.286,0.299,0.321
3,2025-02-18 19:00:00+00:00,-0.522000,69.000000,-5.511998,-3.712921,0.0,0.0,0.0,0.0,0.0,...,-0.522,-0.372,-0.122,0.028,2.628,0.283,0.284,0.286,0.299,0.321
4,2025-02-18 18:00:00+00:00,0.028000,67.000000,-5.372924,-3.147840,0.0,0.0,0.0,0.0,0.0,...,-0.422,-0.172,-0.122,0.028,2.628,0.282,0.283,0.286,0.299,0.321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12883,2023-09-01 03:00:00+00:00,12.282001,93.616058,11.282001,10.968299,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12884,2023-09-01 02:00:00+00:00,12.432000,92.698418,11.282001,11.042597,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12885,2023-09-01 01:00:00+00:00,12.732000,91.496857,11.382000,11.339030,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12886,2023-09-01 00:00:00+00:00,12.832000,90.899940,11.382000,11.366271,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

cols = ['date', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m', 'pressure_msl', 'relative_humidity_2m']

df_air = df_omw[cols]



In [ ]:
import time

today = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
one_week_before = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time() - 60*60*24*7))
output_filename = 'subset_output.nc'

# Get ALL Timeseries data from FROST-Server
server = FrostServer(thing="Things(3)")
all_observations = server.get_all_observations()
df_water_temperature = pd.DataFrame(all_observations)
df_water_temperature["phenomenonTime"] = pd.to_datetime(df_water_temperature["phenomenonTime"])
df_water_temperature["resultTime"] = pd.to_datetime(df_water_temperature["resultTime"])
df_water_temperature["phenomenonTime"] = df_water_temperature["phenomenonTime"].dt.round('h') # round phenomenonTime to YYYY-MM-DD HH:MM:SS
df_water_temperature["result"] = df_water_temperature["result"].astype(float)
df_water_temperature = df_water_temperature.loc[:, ['phenomenonTime', 'result']].drop_duplicates()

# Get position data of T-Box
position_url = server.get_position_url()
content = server.get_content(position_url)
coordinates_t_box =  content['value'][0]['location']['coordinates']
longitude, latitude = coordinates_t_box[0], coordinates_t_box[1]

thing_name = server.get_thing_name()


start_datetime = df_water_temperature["phenomenonTime"].min()
start_datetime = start_datetime.strftime('%Y-%m-%d %H:%M:%S')
start_datetime = start_datetime.split(' ')[0]

end_datetime = end_datetime.split(' ')[0]

# get weather data
omw = OpenMeteoWeather(latitude=latitude, 
                longitude=longitude, 
                start_date=start_datetime, 
                end_date=end_datetime.split(' ')[0]
                )
df_omw = omw.get_weather_dataframe()

cols = ['date', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m', 'pressure_msl', 'relative_humidity_2m']
df_omw = df_omw[cols]
df_omw["date"] = pd.to_datetime(df_omw["date"])

#merge weather data with water temperature data
df_merged = pd.merge(df_water_temperature, df_omw, left_on='phenomenonTime', right_on='date', how='left')
display(df_merged)
display(df_merged.info())


,phenomenonTime,result,date,wind_speed_10m,wind_direction_10m,temperature_2m,pressure_msl,relative_humidity_2m
0,2024-04-17 01:00:00+00:00,18.00,2024-04-17 01:00:00+00:00,8.121970,282.804260,3.982000,1007.099976,86.801567
1,2024-04-17 02:00:00+00:00,17.75,2024-04-17 02:00:00+00:00,6.489992,303.690094,2.632000,1006.900024,91.141571
2,2024-04-17 03:00:00+00:00,17.56,2024-04-17 03:00:00+00:00,8.161764,311.423584,1.532000,1006.900024,90.081215
3,2024-04-19 14:00:00+00:00,19.75,2024-04-19 14:00:00+00:00,15.629971,298.926331,9.282001,998.200012,76.338776
4,2024-04-19 17:00:00+00:00,19.68,2024-04-19 17:00:00+00:00,27.859905,32.868675,6.032000,1001.799988,88.542221
...,...,...,...,...,...,...,...,...
6735,2025-02-18 11:00:00+00:00,2.43,2025-02-18 11:00:00+00:00,6.480000,270.000000,-0.722000,1031.000000,55.000000
6736,2025-02-18 12:00:00+00:00,2.31,2025-02-18 12:00:00+00:00,4.735060,278.746063,0.628000,1030.800049,48.000000
6737,2025-02-18 13:00:00+00:00,2.25,2025-02-18 13:00:00+00:00,2.036468,315.000092,1.428000,1030.500000,48.000000
6738,2025-02-18 14:00:00+00:00,2.31,2025-02-18 14:00:00+00:00,1.080000,360.000000,1.778000,1030.400024,48.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6740 entries, 0 to 6739
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   phenomenonTime        6740 non-null   datetime64[ns, UTC]
 1   result                6740 non-null   float64            
 2   date                  6740 non-null   datetime64[ns, UTC]
 3   wind_speed_10m        6718 non-null   float32            
 4   wind_direction_10m    6718 non-null   float32            
 5   temperature_2m        6718 non-null   float32            
 6   pressure_msl          6718 non-null   float32            
 7   relative_humidity_2m  6718 non-null   float32            
dtypes: datetime64[ns, UTC](2), float32(5), float64(1)
memory usage: 289.7 KB


None